# Handle Tensorflow Implementations


In [11]:
import pathlib
import numpy as np
import tensorflow as tf

In [12]:
seizure_type = "combined"
model_type = "default"

Import dataset for signature


In [13]:
main_folder_path = "melspectrograms/"
path = main_folder_path + seizure_type + "/"

if seizure_type == "combined":
    path = main_folder_path + "absence" + "/"
    train_x_absence = np.load(f"{path}/train_absence_x.npy")
    train_y_absence = np.load(f"{path}/train_absence_y.npy")

    path = main_folder_path + "tonic-clonic" + "/"
    train_x_tonic_clonic = np.load(f"{path}/train_tonic-clonic_x.npy")
    train_y_tonic_clonic = np.load(f"{path}/train_tonic-clonic_y.npy")

    for i, v in enumerate(train_x_tonic_clonic):
        if train_y_tonic_clonic[i] == 1:
            train_y_tonic_clonic[i] = 2

    path = main_folder_path + "general" + "/"
    train_x_general = np.load(f"{path}/train_general_x.npy")
    train_y_general = np.load(f"{path}/train_general_y.npy")

    for i, v in enumerate(train_x_general):
        if train_y_general[i] == 1:
            train_y_general[i] = 3

    train_x = np.concatenate((train_x_absence, train_x_tonic_clonic, train_x_general))
    train_y = np.concatenate((train_y_absence, train_y_tonic_clonic, train_y_general))
else:
    train_x = np.load(f"{path}/train_{seizure_type}_x.npy")
    train_y = np.load(f"{path}/train_{seizure_type}_y.npy")

Import Model


In [14]:
pb_model_path = f"model/tf_saved_models/{seizure_type}_2_model"
print(pb_model_path)

model/tf_saved_models/combined_2_model


In [15]:
file_path = f"model/tflite_models/"
tflite_models_dir = pathlib.Path(file_path)

In [16]:
# load model
model = tf.saved_model.load(pb_model_path)

for i in train_x:
    print(i.shape)
    i = np.expand_dims(i, axis=0).astype(np.float32)
    i = np.expand_dims(i, axis=0).astype(np.float32)
    print(i.shape)
    break

(40, 26)
(1, 1, 40, 26)


In [17]:
print("Converting to TFLite...")

# Load File
converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model(pb_model_path)


def representative_dataset():
    for i in train_x:
        i = np.expand_dims(i, axis=0).astype(np.float32)
        i = np.expand_dims(i, axis=0).astype(np.float32)
        yield [i]


# # Apply optimisations
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert
tflite_model = converter.convert()

# Save
tflite_model_quant_file = tflite_models_dir / f"{seizure_type}_2.tflite"
tflite_model_quant_file.write_bytes(tflite_model)

# https://www.kaggle.com/code/jarvisai7/tensorflow-pytorch-to-tflite

Converting to TFLite...
INFO:tensorflow:Restoring parameters from model/tf_saved_models/combined_2_model\variables\variables


INFO:tensorflow:Restoring parameters from model/tf_saved_models/combined_2_model\variables\variables


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}


INFO:tensorflow:Restoring parameters from model/tf_saved_models/combined_2_model\variables\variables


...

INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}


INFO:tensorflow:Restoring parameters from model/tf_saved_models/combined_2_model\variables\variables


INFO:tensorflow:Restoring parameters from model/tf_saved_models/combined_2_model\variables\variables


53984

In [18]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ""

    # Create header guard
    c_str += "#ifndef " + var_name.upper() + "_H\n"
    c_str += "#define " + var_name.upper() + "_H\n\n"

    # Add array length at top of file
    c_str += "\nunsigned int " + var_name + "_len = " + str(len(hex_data)) + ";\n"

    # Declare C variable
    c_str += "unsigned char " + var_name + "[] = {"
    hex_array = []
    for i, val in enumerate(hex_data):

        # Construct string from hex
        hex_str = format(val, "#04x")

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ","
        if (i + 1) % 12 == 0:
            hex_str += "\n "
        hex_array.append(hex_str)

    # Add closing brace
    c_str += "\n " + format(" ".join(hex_array)) + "\n};\n\n"

    # Close out header guard
    c_str += "#endif //" + var_name.upper() + "_H"

    return c_str

In [19]:
with open(f"model/model_c_files/{seizure_type}_2" + ".h", "w") as file:
    file.write(hex_to_c_array(tflite_model, f"model/model_c_files/{seizure_type}_2"))

In [68]:
# https://gist.github.com/ShawnHymel/79237fe6aee5a3653c497d879f746c0c